In [1]:
using Flux.Tracker
using Flux
using Flux:throttle,glorot_uniform
using BSON:@save
using BSON:@load
using Base.Iterators: repeated
using Flux:@treelike
using PyPlot
using FastGaussQuadrature
using SparseGrids
using LinearAlgebra
using Flux: @epochs
using DataFrames
using CSV
using Sobol
using Pkg; Pkg.activate("cuda"); Pkg.instantiate()
using CuArrays
using ForwardDiff
using CUDAnative
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float32,1}, ::Val{2}) = x*x
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float64,1}, ::Val{2}) = x*x

f(x)=2*π^2 .*sum(cos.(π*x[i,:]') for i=1:16, dims=1)
acti(x)=@. x/(1+CUDAnative.exp(-x))


struct Block{F,S,T}
  W1::S
  W2::S
  b1::T
  b2::T
  σ::F
end

Block(W1 , W2, b1, b2) = Block(W1,W2 ,b1 ,b2, identity)

function Block(in1::Integer,in2::Integer, out1::Integer,out2::Integer, σ = identity;
  initW1 = glorot_uniform, initW2 =glorot_uniform, initb1 = zeros,initb2=zeros)
  return Block(param(initW1(out1, in1)),param(initW2(out2,in2)), param(initb1(out1)), param(initb2(out2)),σ)
end

@treelike Block

function (a::Block)(x)
  W1,W2, b1,b2, σ = a.W1,a.W2,a.b1, a.b2, a.σ
  σ.(W2 *σ.(W1*x .+ b1) .+ b2) .+ x
end





M=Chain(
Dense(16,48),
Block(48,48,48,48,acti),
Block(48,48,48,48,acti),
Block(48,48,48,48,acti),
Block(48,48,48,48,acti),
Block(48,48,48,48,acti),
Dense(48,1)
)|>gpu

function lossFD(N)
  notes=rand(16,N)
    for i=1:N
      notes[:,i]=next!(inner)
    end
  notes =notes |>gpu
  d=rand(1)/100 |>gpu
  losses=0
  MMM=M(notes)
  for i=1:16
    l1=zeros(16)
    l1[i]=1
    l1 = l1 |>gpu
    losses  += (sum(0.5*((M(notes .+ d.*l1) .- MMM)./d).^2)/N)[1]
  end
  losses += (sum(0.5*(π^2 * (MMM).^2))/N)[1]
  losses += (sum(-f(notes) .*MMM)/N)[1]
  return losses
end


function lossBC(β)
  d=(rand(1)/100)|>gpu
  point=zeros(16,3200)
  p=rand(15,100)
    for i=1:100
      p[:,i]=next!(BC)
    end
  for i=1:16
    point[:,200*(i-1)+1:200*(i-1)+100]=vcat(p[1:i-1,:],zeros(100)',p[i:end,:])
    point[:,200*(i-1)+101:200*i]=vcat(p[1:i-1,:], ones(100)',p[i:end,:])
  end
    point=point|>gpu
  losses=0
  for i=1:16
    l1=zeros(16)
    l1[i]=1
    l1=l1|>gpu
    losses +=β*(sum((( 1/2*M(point[:,200*(i-1)+1:200*i] .+ d.*l1) .- 1/2*M(point[:,200*(i-1)+1:200*i] .- d.*l1))./d).^2)/100)[1]
  end
  return losses
end

function loss1(N)
  return lossBC(50)+lossFD(N)
end


function loss_true(xx=1,yy=1,xx1=1,xx2=2)
  points=zeros(16,5000)
    for i=1:5000
      points[:,i]=next!(truep)
    end
    points=points  |>gpu
  F_true(x)= sum(cos.(pi*x[i,:]') for i=1:16, dims=1)
  errors=sqrt(sum((M(points)-F_true(points)).^2)/5000)
  return errors
end


inner=SobolSeq(16)
truep=SobolSeq(16)
BC=SobolSeq(15)


function test(N;traintime=2000)
  errdf=DataFrame(time=Int[],absolute_error=Float64[],loss_FD=Float64[],loss_BC=Float64[])
  cntr=1
  evalcb = function()
    loss_true1=loss_true()
    loss_FD = lossFD(N)
    loss_BC =lossBC(1)
    push!(errdf,[Tracker.data(cntr),Tracker.data(loss_true1),Tracker.data(loss_FD),Tracker.data(loss_BC)])
    cntr+=1
    if cntr%1000==0
      @show(cntr,loss_FD,loss_true1,loss_BC)
      CSV.write("$(N)Sobol16Dexm2.csv",errdf)
      @save "$(N)Sobolpoint16Dexm2m.bson" M
      weights=Tracker.data.(params(M))
      @save "$(N)Sobolpoint16Dexm2w.bson" weights
    end
  end
  θ=Flux.params(M)
  opt=ADAM()
  dataset=[(N) for i=1:traintime]
  Flux.train!(loss1, params(M), zip(dataset), opt, cb=evalcb)
  return errdf
end


Activating new environment at `~/exm16sobol/cuda/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


test (generic function with 1 method)

In [2]:
errdf=test(500,traintime=30000)
for i=1:4
    N=[1000 2000 5000 10000]
    errdf=test(N[i],traintime=50000)
end

cntr = 1000
loss_FD = -17.974048347011617 (tracked)
loss_true1 = 2.4684114f0 (tracked)
loss_BC = 0.06216681137854849 (tracked)
cntr = 2000
loss_FD = -43.882138131741954 (tracked)
loss_true1 = 1.9696076f0 (tracked)
loss_BC = 0.05831716367461816 (tracked)
cntr = 3000
loss_FD = -50.4986452334464 (tracked)
loss_true1 = 1.7000027f0 (tracked)
loss_BC = 0.04609143266573256 (tracked)
cntr = 4000
loss_FD = -59.59206986258455 (tracked)
loss_true1 = 1.4070774f0 (tracked)
loss_BC = 0.0543083470717466 (tracked)
cntr = 5000
loss_FD = -61.677232013983996 (tracked)
loss_true1 = 1.2648804f0 (tracked)
loss_BC = 0.05978169174239849 (tracked)
cntr = 6000
loss_FD = -67.0029590418435 (tracked)
loss_true1 = 1.2036598f0 (tracked)
loss_BC = 0.0462776498083137 (tracked)
cntr = 7000
loss_FD = -65.87589404996483 (tracked)
loss_true1 = 1.0058017f0 (tracked)
loss_BC = 0.10666524846748131 (tracked)
cntr = 8000
loss_FD = -68.23466461215082 (tracked)
loss_true1 = 0.91317564f0 (tracked)
loss_BC = 0.0570081507804919 (tr

cntr = 35000
loss_FD = -78.22571575103399 (tracked)
loss_true1 = 0.040331755f0 (tracked)
loss_BC = 0.004210998311508523 (tracked)
cntr = 36000
loss_FD = -79.58185317787985 (tracked)
loss_true1 = 0.040282983f0 (tracked)
loss_BC = 0.004215257264734108 (tracked)
cntr = 37000
loss_FD = -78.22165101571332 (tracked)
loss_true1 = 0.040092018f0 (tracked)
loss_BC = 0.0044624606724925995 (tracked)
cntr = 38000
loss_FD = -79.3008444717359 (tracked)
loss_true1 = 0.03615416f0 (tracked)
loss_BC = 0.006552233890523587 (tracked)
cntr = 39000
loss_FD = -79.33346216601086 (tracked)
loss_true1 = 0.037507396f0 (tracked)
loss_BC = 0.007014178758288427 (tracked)
cntr = 40000
loss_FD = -79.76860141210119 (tracked)
loss_true1 = 0.034530777f0 (tracked)
loss_BC = 0.005634881491312128 (tracked)
cntr = 41000
loss_FD = -81.44513883897102 (tracked)
loss_true1 = 0.035133835f0 (tracked)
loss_BC = 0.004902667100354495 (tracked)
cntr = 42000
loss_FD = -79.29110745428932 (tracked)
loss_true1 = 0.03642576f0 (tracked)
los

cntr = 49000
loss_FD = -78.55881449048795 (tracked)
loss_true1 = 0.03128615f0 (tracked)
loss_BC = 0.004138519837312476 (tracked)
cntr = 50000
loss_FD = -79.29550344449954 (tracked)
loss_true1 = 0.03170617f0 (tracked)
loss_BC = 0.0030849479357008855 (tracked)
cntr = 1000
loss_FD = -79.05662378908733 (tracked)
loss_true1 = 0.03228495f0 (tracked)
loss_BC = 0.0010231509568185322 (tracked)
cntr = 2000
loss_FD = -78.81081370804083 (tracked)
loss_true1 = 0.0314259f0 (tracked)
loss_BC = 0.008904633666002183 (tracked)
cntr = 3000
loss_FD = -78.7896439552654 (tracked)
loss_true1 = 0.029316446f0 (tracked)
loss_BC = 0.0019531249671152278 (tracked)
cntr = 4000
loss_FD = -79.00087124553441 (tracked)
loss_true1 = 0.027978493f0 (tracked)
loss_BC = 0.0022330771465018653 (tracked)
cntr = 5000
loss_FD = -78.98149897796924 (tracked)
loss_true1 = 0.027601918f0 (tracked)
loss_BC = 0.0015010452161838004 (tracked)
cntr = 6000
loss_FD = -79.14826964482272 (tracked)
loss_true1 = 0.033168122f0 (tracked)
loss_BC 

cntr = 13000
loss_FD = -78.86900652662824 (tracked)
loss_true1 = 0.030102829f0 (tracked)
loss_BC = 0.002008449427493472 (tracked)
cntr = 14000
loss_FD = -78.96482361529706 (tracked)
loss_true1 = 0.03729589f0 (tracked)
loss_BC = 0.007403324081151471 (tracked)
cntr = 15000
loss_FD = -78.8850595013268 (tracked)
loss_true1 = 0.028446198f0 (tracked)
loss_BC = 0.0016877672390410018 (tracked)
cntr = 16000
loss_FD = -78.92591055628795 (tracked)
loss_true1 = 0.026352936f0 (tracked)
loss_BC = 0.0010760490536615575 (tracked)
cntr = 17000
loss_FD = -78.92166490113037 (tracked)
loss_true1 = 0.027237039f0 (tracked)
loss_BC = 0.0018162588693787119 (tracked)
cntr = 18000
loss_FD = -73.45197391552452 (tracked)
loss_true1 = 0.7554103f0 (tracked)
loss_BC = 0.1685120042777421 (tracked)
cntr = 19000
loss_FD = -79.10220744629734 (tracked)
loss_true1 = 0.027126025f0 (tracked)
loss_BC = 0.002626731648486678 (tracked)
cntr = 20000
loss_FD = -78.8488103216706 (tracked)
loss_true1 = 0.025680479f0 (tracked)
loss_